In [ ]:
%pylab inline

In [ ]:
import tensorflow as tf
from tensorflow.contrib import keras
from tensorflow.contrib.keras.python.keras import models, losses
from tensorflow.contrib.keras.python.keras.layers.core import Dense,Reshape,Flatten, Dropout
from tensorflow.contrib.keras.python.keras.layers.pooling import MaxPool2D
from tensorflow.contrib.keras.python.keras.layers.convolutional import ZeroPadding2D, Conv2D
from tensorflow.contrib.keras.python.keras.layers.merge import Concatenate, Add
from tensorflow.contrib.keras.python.keras.layers.recurrent import LSTM, GRU
from tensorflow.contrib.keras.python.keras.layers.pooling import MaxPool2D
from tensorflow.contrib.keras.python.keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# basic
import os
import random
import json
import copy
import pandas as pd
import seaborn as sns
sns.set(style="darkgrid")

# geo-related
from osgeo import gdal
import geopandas as gpd
import folium
from folium import plugins

# scikit-learn
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc

from rscnn_utils import utils

In [ ]:
seed = 7
random.seed(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [ ]:
def plot_roc_curve(y_true, y_pred_prob):
   # Compute micro-average ROC curve and ROC area
    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
    # fpr, tpr, _ = roc_curve(y_true_ohl.ravel(), y_pred_prob.ravel())

    roc_auc = auc(fpr, tpr) 
    
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
folder = './buildData/'

ht = gdal.Open(os.path.join(folder,'mosaic_post.tif'))
gt = ht.GetGeoTransform()

buildings_df = gpd.read_file(os.path.join(folder,'building_post_id_join_reclassified.shp'))
buildings_df['image_id']='mosaic_post'

In [ ]:
buildings_df['raw_index'] = buildings_df.index
print(buildings_df.crs)
buildings_df.head()

In [ ]:
buildings_df['Destruction'] = buildings_df.apply(lambda x: 'Destruction' if x['Damage_ID'] =='GRADE 5 Destruction' else 'Non-destrctuion', axis=1)

In [ ]:
utils = utils()

In [ ]:
chips_dim = buildings_df.apply(lambda poly: pd.Series(utils.chip_dim(poly['geometry'],poly['image_id'])),axis=1)

In [ ]:
buildings_df = pd.concat([buildings_df, chips_dim], axis=1, join_axes=[chips_dim.index])
buildings_df.shape

In [ ]:
min_dim = 10
max_dim = 100

chips_dim_min = chips_dim.apply(lambda chip: chip[0]>min_dim and chip[1]>min_dim,axis=1)
chips_dim_max = chips_dim.apply(lambda chip: chip[0]<max_dim and chip[1]<max_dim,axis=1)

chips_filter_dim = chips_dim_min & chips_dim_max
print (chips_filter_dim.describe())

#### post- Data

In [ ]:
buildings_filter = buildings_df[chips_filter_dim]
buildings_filter.reset_index(drop=True,inplace=True)

In [ ]:
chip_uni_ = []
for x in range(len(buildings_filter)):
    chip = utils.vector_clip_raster(buildings_filter.loc[x,'geometry'],buildings_filter.loc[x,'image_id'])         
    chip = np.transpose(chip,(1,2,0))   ################## transpose ########
    chip = utils.uniform_chips(chip,resize=False,max_dim=max_dim)
    chip_uni_.append(chip)

In [ ]:
chip_uni = np.array(chip_uni_)

chip_uni.nbytes/10**6

#### pre- Data

In [ ]:
pre_buildings_filter = copy.deepcopy(buildings_filter)
pre_buildings_filter['image_id']='mosaic_pre_registered'

In [ ]:
pre_chip_uni_ = []
for x in range(len(pre_buildings_filter)):
    chip = utils.vector_clip_raster(pre_buildings_filter.loc[x,'geometry'],pre_buildings_filter.loc[x,'image_id'])         
    chip = np.transpose(chip,(1,2,0))   ################## transpose ########
    chip = utils.uniform_chips(chip,resize=False,max_dim=max_dim)
    pre_chip_uni_.append(chip)
    
pre_chip_uni = np.array(pre_chip_uni_)
pre_chip_uni.nbytes/10**6

#### Diff (pre- & post-)

In [ ]:
diff = chip_uni-pre_chip_uni

### Modelling

#### CNN-diff

In [ ]:
diff_X_train, diff_X_test, diff_y_train, diff_y_test = model_selection.train_test_split(
                    diff, pre_buildings_filter['Destruction'].values, test_size=0.33, random_state=42)

In [ ]:
positive = 0
negative = 1

# one-hot-label
y_train_digits = [negative if bild=='Destruction' else positive for bild in diff_y_train]
y_train_ohl = keras.utils.to_categorical(y_train_digits, num_classes=2)

y_true = [negative if bild=='Destruction' else positive for bild in diff_y_test]

print(diff_y_train[0])
print(y_train_digits[0])
print(y_train_ohl[0])

In [ ]:
from squeezenet import smallSqueezeNet

In [ ]:
input_shape = (max_dim,max_dim,3)

def sqzmodel(optimizer='adam', init='he_uniform',summary=False):
    sqzNet = smallSqueezeNet(input_shape=input_shape, init=init)
    sqzNet.compile(optimizer=optimizer, loss = losses.categorical_crossentropy, metrics=['accuracy'])
    if summary:
        sqzNet.summary()
    return sqzNet

In [ ]:
kModel = KerasClassifier(build_fn=sqzmodel, batch_size=128, epochs=50, verbose=1)
hist = kModel.fit(x=diff_X_train, y=y_train_ohl,validation_split=0.1)

In [ ]:
y_pred=kModel.model.predict(diff_X_test)
y_pred = np.argmax(y_pred,axis=1)

In [ ]:
cm = confusion_matrix(y_true, y_pred)
print (cm)

print(accuracy_score(y_true, y_pred))
print (precision_score(y_true,y_pred))
print (recall_score(y_true,y_pred))
print (f1_score(y_true,y_pred))

In [ ]:
print(classification_report(y_true, y_pred, target_names=['no Destruction','Destruction']))

In [ ]:
y_pred_prob = kModel.model.predict(diff_X_test)
plot_roc_curve(y_true, y_pred_prob[:,1])

### SqueezeNet + GRU/LSTM

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
                    chip_uni, buildings_filter['Destruction'].values, test_size=0.33, random_state=42)

In [ ]:
pre_X_train, pre_X_test, pre_y_train, pre_y_test = model_selection.train_test_split(
                    pre_chip_uni, pre_buildings_filter['Destruction'].values, test_size=0.33, random_state=42)

In [ ]:
from tensorflow.contrib.keras.python.keras.layers import Input

def squeezeLSTM(input_shape=input_shape, classes=128, summary=False):
    pre_input = Input(shape=input_shape)
    left = smallSqueezeNet(input_tensor=pre_input,classes=128,return_features=True)
    post_input = Input(shape=input_shape)
    right = smallSqueezeNet(input_tensor=post_input,classes=128,return_features=True)
    x = concatenate([left, right], axis=1) # K.image_data_format() == 'channels_last'
    x = Reshape((-1,128))(x)
    x = GRU(128, dropout=0.5, return_sequences=False)(x)
    x = Dense(48, activation='relu')(x)
    output = Dense(2, activation='sigmoid', name='main_output')(x) # sigmoid

    model = Model(inputs=[pre_input, post_input], outputs=[output])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    if summary:
        model.summary()
    return model

In [ ]:
kModel = KerasClassifier(build_fn=squeezeLSTM, batch_size=128, epochs=50, verbose=1)
hist = kModel.fit(x=[pre_X_train,X_train], y=y_train_ohl,validation_split=0.1)

In [ ]:
y_pred = kModel.model.predict([pre_X_test, X_test])
y_pred = np.argmax(y_pred,axis=1)

In [ ]:
cm = confusion_matrix(y_true, y_pred)
print (cm)
print (accuracy_score(y_true,y_pred))
print (precision_score(y_true,y_pred))
print (recall_score(y_true,y_pred))
print (f1_score(y_true,y_pred))

In [ ]:
print(classification_report(y_true, y_pred, target_names=['no Destruction','Destruction']))

In [ ]:
y_pred_prob = kModel.model.predict([pre_X_test, X_test])
plot_roc_curve(y_true, y_pred_prob[:,1])